# Using the CPR search API
This brief tutorial will show you how to use the CPR Search API. We've provided some helper functions but you should note that these are WIP and there may be bugs or issues that we don't know about. 

So before we get into the python, lets go through some details of the API itself so you can make queries without them. 

You can call the API using the requests library like this:

In [6]:
API_HOST = "https://app.climatepolicyradar.org"
import requests

url = f"{API_HOST}/api/v1/searches"
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}
data = '{"query_string": "Energy Prices"}'

response = requests.post(url, headers=headers, data=data)
results = response.json()
results['total_family_hits'] # How many families did we find?

1370

Nice and easy. Except that you might not be expecting the word "Families". This is what we call a collection of documents related to a law or policy. A family has a single main document that is the text of the law or policy, but there may be amendments, official translations, or other documents that we also have in our database for that law/policy. We collect these all as a family, which has potentially many documents under it. 

You can adjust the search query quite a lot via the API. For example, in the following request, we are searching for documents that contain text similar to "renewable energy" from the USA, limiting to 10 hits per document, 5 total results per page, and using our semantic search. If you set exact_match to "True", you'd get only text that exactly matches "renewable energy". You can find out more about the parameters for the API in the api-docs.md file

In [7]:

API_HOST = "https://app.climatepolicyradar.org"
import requests

url = f"{API_HOST}/api/v1/searches"
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}
data = '{"query_string": "renewable energy", "exact_match": "False", "max_passages_per_doc": 10, "sort_order": "descending", "limit": 5, "keyword_filters": {"countries": ["united-states-of-america"]}}'

response = requests.post(url, headers=headers, data=data)
results = response.json()
results["families"][0]["family_name"]

'United States of America. Biennial Reports (BR). BR 1. National Communication (NC). NC 6.'

We've provided some utility functions for you to make querying the API easier -- although as mentioned before, these are early WIP. Lets have a look at how to use them. You'll have to ensure the CPR SDK python package is installed, using ```pip install cpr_sdk```

In [8]:
from utils import APISearchAdapter, search_response_to_markdown
from cpr_sdk.models.search import SearchParameters
from IPython.display import display, Markdown

# Create an instance of the APISearchAdapter with the API URL
search_adapter = APISearchAdapter(api_url="https://app.climatepolicyradar.org")

# Define search parameters
search_params = SearchParameters(
    query_string="electric vehicles",
    limit=1 # We'll keep the limit to 1 so there's not too many results to scroll through
)

# Perform the search
search_response = search_adapter.search(search_params)

# Have a look at the results
display(Markdown(search_response_to_markdown(search_response)))

| Family Title | Geography | PDF URL | Matched Passage |
| --- | --- | --- | --- |
| United Republic of Tanzania. National Communication (NC). NC 2. | TZA | https://app.climatepolicyradar.org/document/united-republic-of-tanzania-national-communication-nc-nc-2_be66 | · Electricity-Hybrid Electric Vehicles, Electric Vehicles and Efficiency |
| United Republic of Tanzania. National Communication (NC). NC 2. | TZA | https://app.climatepolicyradar.org/document/united-republic-of-tanzania-national-communication-nc-nc-2_be66 | vehicle |

See how there are two matches of text, but we only asked for a limit of 1? That's because the limit is for the number of families, not the number of search matches within the document text. We treat each separately. 

We can use Filters this way as well! Lets search for legislative documents from the USA:

In [9]:
from cpr_sdk.models.search import SearchParameters, Filters

search_adapter = APISearchAdapter(api_url="https://app.climatepolicyradar.org")

# Define search filters
filters = Filters(
    family_geography=["United States of America"],
    family_category=["Legislative"], # Or "Executive" or "UNFCCC"
)

search_params = SearchParameters(
    query_string="renewable energy",
    filters=filters,
    limit=1
)

search_response = search_adapter.search(search_params)
 
display(Markdown(search_response_to_markdown(search_response)))


| Family Title | Geography | PDF URL | Matched Passage |
| --- | --- | --- | --- |
| Food, Conservation, and Energy Act of 2008 (revised 2014) - Title IX-Renewable Energy Provisions | USA | https://app.climatepolicyradar.org/document/food-conservation-and-energy-act-of-2008-revised-2014-title-ix-renewable-energy-provisions_1709 | "(13) RENEWABLE ENERGY .- The term 'renewable energy' means energy derived from- |
| Food, Conservation, and Energy Act of 2008 (revised 2014) - Title IX-Renewable Energy Provisions | USA | https://app.climatepolicyradar.org/document/food-conservation-and-energy-act-of-2008-revised-2014-title-ix-renewable-energy-provisions_1709 | "SEC. 1408B. RENEWABLE ENERGY COMMITTEE. |
| Food, Conservation, and Energy Act of 2008 (revised 2014) - Title IX-Renewable Energy Provisions | USA | https://app.climatepolicyradar.org/document/food-conservation-and-energy-act-of-2008-revised-2014-title-ix-renewable-energy-provisions_1709 | Sec. 7104. Renewable energy committee. |
| Food, Conservation, and Energy Act of 2008 (revised 2014) - Title IX-Renewable Energy Provisions | USA | https://app.climatepolicyradar.org/document/food-conservation-and-energy-act-of-2008-revised-2014-title-ix-renewable-energy-provisions_1709 | SEC. 7104. RENEWABLE ENERGY COMMITTEE. |
| Food, Conservation, and Energy Act of 2008 (revised 2014) - Title IX-Renewable Energy Provisions | USA | https://app.climatepolicyradar.org/document/food-conservation-and-energy-act-of-2008-revised-2014-title-ix-renewable-energy-provisions_1709 | "(iii) agriculture-based renewable energy resources. |
| Food, Conservation, and Energy Act of 2008 (revised 2014) - Title IX-Renewable Energy Provisions | USA | https://app.climatepolicyradar.org/document/food-conservation-and-energy-act-of-2008-revised-2014-title-ix-renewable-energy-provisions_1709 | "(i) renewable energy development and trans- mission; |
| Food, Conservation, and Energy Act of 2008 (revised 2014) - Title IX-Renewable Energy Provisions | USA | https://app.climatepolicyradar.org/document/food-conservation-and-energy-act-of-2008-revised-2014-title-ix-renewable-energy-provisions_1709 | SEC. 6108. ELECTRIC LOANS FOR RENEWABLE ENERGY. |
| Food, Conservation, and Energy Act of 2008 (revised 2014) - Title IX-Renewable Energy Provisions | USA | https://app.climatepolicyradar.org/document/food-conservation-and-energy-act-of-2008-revised-2014-title-ix-renewable-energy-provisions_1709 | "(1) improving on-farm renewable energy production; |
| Food, Conservation, and Energy Act of 2008 (revised 2014) - Title IX-Renewable Energy Provisions | USA | https://app.climatepolicyradar.org/document/food-conservation-and-energy-act-of-2008-revised-2014-title-ix-renewable-energy-provisions_1709 | "SEC. 317. ELECTRIC LOANS FOR RENEWABLE ENERGY. |


There are limits to how many results you can get from our search API. If you need to do anything that requires a lot of requests or doing operations over many documents at once, we recommend you use the downloadable dataset instead. Please be responsible with the amount you call our API!

You can use the API to get the next page of family results, or the next page of matches within a document.

In [16]:
# Define search parameters
search_params = SearchParameters(
    query_string="tax",
    limit=1
)

# Perform the search
page_1 = search_adapter.search(search_params)

# Have a look at the results
display(Markdown(search_response_to_markdown(page_1)))

# Get the "continuation token" from the search response, which will let us tell the API to give us the next page of results
search_params.continuation_tokens = [page_1.continuation_token]
page_2 = search_adapter.search(search_params)
display(Markdown(search_response_to_markdown(page_2)))

| Family Title | Geography | PDF URL | Matched Passage |
| --- | --- | --- | --- |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Taxes/tax relief (including carbon taxes, energy taxes and congestion taxes) |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | tax |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Italy. 36 pp. https://www.oecd.org/tax/tax-policy/tax-policy-and-climate- |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Carbon taxes |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Border tax adjustment |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | carbon taxes 270 |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | carbon taxes 1386 |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | carbon taxes 466 |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | taxes 1292-1293 |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | see also carbon taxes |

| Family Title | Geography | PDF URL | Matched Passage |
| --- | --- | --- | --- |
| Carbon Tax Act 15 | ZAF | https://app.climatepolicyradar.org/document/carbon-tax-act-15_0872 | Administration, tax period and payment of tax |
| Carbon Tax Act 15 | ZAF | https://app.climatepolicyradar.org/document/carbon-tax-act-15_0872 | Administration, tax period and payment of tax |
| Carbon Tax Act 15 | ZAF | https://app.climatepolicyradar.org/document/carbon-tax-act-15_0872 | Tax base |
| Carbon Tax Act 15 | ZAF | https://app.climatepolicyradar.org/document/carbon-tax-act-15_0872 | Tax period |
| Carbon Tax Act 15 | ZAF | https://app.climatepolicyradar.org/document/carbon-tax-act-15_0872 | Rate of tax |
| Carbon Tax Act 15 | ZAF | https://app.climatepolicyradar.org/document/carbon-tax-act-15_0872 | Payment of tax |
| Carbon Tax Act 15 | ZAF | https://app.climatepolicyradar.org/document/carbon-tax-act-15_0872 | Payment of tax |
| Carbon Tax Act 15 | ZAF | https://app.climatepolicyradar.org/document/carbon-tax-act-15_0872 | 4. Tax base |
| Carbon Tax Act 15 | ZAF | https://app.climatepolicyradar.org/document/carbon-tax-act-15_0872 | Administration Tax period |

In [17]:
# Using continuation tokens to get to more matches for a document
# Define search parameters
search_params = SearchParameters(
    query_string="tax",
    limit=1
)

# Perform the search
search_response = search_adapter.search(search_params)

# Each family in the search results has a continuation token. This is what tells the API "more passages from THIS family please"
passage_token = search_response.families[0].continuation_token

display(Markdown(search_response_to_markdown(search_response)))

# Use the continuation token to redo the search: give it the CURRENT search continuation token and the passage token we got above 
search_params.continuation_tokens = [search_response.this_continuation_token, passage_token]
more_passages_response = search_adapter.search(search_params)

display(Markdown(search_response_to_markdown(more_passages_response)))

| Family Title | Geography | PDF URL | Matched Passage |
| --- | --- | --- | --- |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Taxes/tax relief (including carbon taxes, energy taxes and congestion taxes) |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | tax |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Italy. 36 pp. https://www.oecd.org/tax/tax-policy/tax-policy-and-climate- |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Carbon taxes |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Border tax adjustment |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | carbon taxes 270 |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | carbon taxes 1386 |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | carbon taxes 466 |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | taxes 1292-1293 |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | see also carbon taxes |

| Family Title | Geography | PDF URL | Matched Passage |
| --- | --- | --- | --- |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Taxes on food products |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | GHG taxes on food |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Taxes on food products |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | see also carbon taxes |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Taxes on food products |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Carbon and environmental taxes |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | GHG taxes on food |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | GHG taxes on food |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | carbon taxes 1384, 1385 |
| Climate Change 2022: Mitigation of Climate Change | XAA | https://app.climatepolicyradar.org/document/climate-change-2022-mitigation-of-climate-change_33ea | Carbon taxes, GHG emissions trading, fossil fuel taxes, tax credits, grants, renewable energy subsidies, fossil fuel subsidy reductions, offsets, R&D subsidies, loan guarantees |